# LocalLab Client Example

This notebook demonstrates how to use the LocalLab client to connect to a running LocalLab server.

In [ ]:
# Install the client package
!pip install locallab-client

### Note: Make sure you have a LocalLab server running!
You need to:
1. Open another Colab notebook
2. Install LocalLab server: `pip install locallab`
3. Start the server with ngrok: 
```python
import os
os.environ["NGROK_AUTH_TOKEN"] = "your_token"
from locallab import start_server
start_server(use_ngrok=True)
```
4. Copy the ngrok URL provided by the server

In [ ]:
import asyncio
from locallab import LocalLabClient

# Replace with your LocalLab server's ngrok URL
SERVER_URL = "https://xxxx-xx-xx-xxx-xx.ngrok.io"  # Get this from the server notebook

# Initialize client
client = LocalLabClient({
    "base_url": SERVER_URL,
    "timeout": 30.0
})

In [ ]:
# Check if server is running
async def check_connection():
    is_healthy = await client.health_check()
    print(f"Server connection: {'OK' if is_healthy else 'Failed'}")

await check_connection()

In [ ]:
# Get available models from server
async def check_models():
    models = await client.list_models()
    print("Available models on server:")
    for model_id, info in models.items():
        print(f"- {model_id}: {info.description}")

await check_models()

In [ ]:
# Basic text generation
async def generate_text():
    response = await client.generate(
        "Tell me a story about a robot learning to paint",
        {"temperature": 0.7}
    )
    print("AI Response:")
    print(response.response)

await generate_text()

In [ ]:
# Chat with the AI
async def chat_with_ai():
    response = await client.chat([
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ])
    print("AI Response:")
    print(response.choices[0].message.content)

await chat_with_ai()

In [ ]:
# Stream responses in real-time
async def stream_response():
    print("AI is thinking...")
    async for token in client.stream_generate("Once upon a time"):
        print(token, end="", flush=True)
    print()

await stream_response()

In [ ]:
# Multiple questions at once
async def batch_questions():
    questions = [
        "What is 2+2?",
        "Who wrote Romeo and Juliet?",
        "What is the speed of light?"
    ]
    
    responses = await client.batch_generate(questions)
    
    print("Batch Responses:")
    for q, a in zip(questions, responses.responses):
        print(f"\nQ: {q}")
        print(f"A: {a}")

await batch_questions()

In [ ]:
# Check server status
async def check_server_status():
    info = await client.get_system_info()
    print("Server Status:")
    print(f"CPU Usage: {info.cpu_usage}%")
    print(f"Memory Usage: {info.memory_usage}%")
    if info.gpu_info:
        print(f"GPU: {info.gpu_info.device}")
    print(f"Active Model: {info.active_model}")

await check_server_status()

In [ ]:
# Clean up
await client.close()